# Initialisation

In [ ]:
!pip install transformers plotly==5.8.0 pyyaml==5.4.1 datasets pytorch-lightning > /dev/null 2>&1
!pip install pytorch_lightning
!pip install datasets
!pip install sentencepiece
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.7 MB/s eta 0:00:00
  Using cached transformers-4.34.1-py3-none-any.whl (7.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-

In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

In [ ]:
# Liens des différents NER pré-programmé
jb = "Jean-Baptiste/camembert-ner-with-dates"
dbmdz = "dbmdz/bert-base-french-europeana-cased"
dslim = "dslim/bert-base-NER"
davlan = "Davlan/bert-base-multilingual-cased-ner-hrl"
test_Adam = "nlpso/m1_ind_layers_ref_ptrn_cmbert_io_level_1"
test_Adam2="nlpso/m1_ind_layers_ref_ptrn_cmbert_io_level_2"

# Initialisation des pipelines
def ner_init(lien):
  ner = pipeline(
      'ner',
      model= AutoModelForTokenClassification.from_pretrained(lien),
      tokenizer= AutoTokenizer.from_pretrained(lien),
      aggregation_strategy= "simple" )
  return ner

In [ ]:
# Chercher un texte généré sur ChatGPT sur Bertrandus qui se balade à Paris
with open("/content/sample_data/Journl d'un Villageois Médiéval en .txt", 'r', encoding='utf-8') as file:
    Bertrandus = file.read()
print(Bertrandus)

Journl d'un Villageois Médiéval en France	10 avril 1365	Aujourd'hui, le soleil s'est levé à l'est, baignant notre modeste hameau de St-Pierre-sous-Bois dans une lueur dorée. Les coqs du village ont entamé leur rituel chant matinal. Jean Dupont, laboureur de la famille Dupont, a quitté sa chaumière pour une nouvelle journée de labeur.	12 avril 1365	La foire de Bourgogne approche à grands pas, et les préparatifs vont bon train. Mon ami Jacques LeForgeron, originaire de la lignée des LeForgeron, m'a demandé de l'aider à préparer des épées pour les chevaliers de la région. Le seigneur local, le comte de Lorraine, Antoine de la Roche, a annoncé qu'il tiendra une joute en l'honneur de son fils, Antoine de la Roche.	15 avril 1365	Jacques LeForgeron et moi avons forgé des épées de qualité, avec une lame tranchante comme un rasoir. Le comte de la Roche a demandé un blason distinctif pour son fils, une épée ornée d'une fleur de lys. Mon fils, Louis Martin, descendant des Martin, m'a rejoint à l'

# Workspace

In [ ]:
ner_jb = ner_init(test_Adam)
ner2=ner_init(test_Adam2)

In [ ]:
def sub_entities(ner, text):
  # Recupere toutes les noms des entites nommés
  entity_array = ner(text)
  word_array = [item['word'] for item in entity_array]

  # List des mots qui composent d'autres entités et qui ne sont pas déjà représenté
  sub_word_array = []

  # Pour chaque mot composé
  for word in word_array:
    if ' ' in word:

      # On l'ajoute ses "sous-mots" si ils ne sont pas déjà dans word_array
      for sub_word in word.split(' '):
        if sub_word not in word_array:
          sub_word_array.append(sub_word)

  # Si la sous entites n'est pas déjà une entité nommée
  for sub_word in sub_word_array:
    sub_entity = ner(sub_word)

    # sub_entity est une list donc extend à la place de append
    entity_array.extend(sub_entity)


  return entity_array

In [ ]:
import re

# Test

In [ ]:
text="Jacques LeForgeron et moi avons forgé des épées de qualité, avec une lame tranchante comme un rasoir. Le comte de la Roche a demandé un blason distinctif pour son fils, une épée ornée d'une fleur de lys. Mon fils, Louis Martin, descendant des Martin, m'a rejoint à l'atelier aujourd'hui. Il se passionne pour les manuscrits anciens et rêve de devenir copiste un jour."

entity2=re.split(r'[.,!?]',text)
entity2=[sentence.strip() for sentence in entity2 if sentence.strip()]
for sentence in entity2:
  print(ner2(sentence))
entity=re.split(r'[.,!?]',text)
entity=[sentence.strip() for sentence in entity if sentence.strip()]
for sentence in entity:
  print(ner_jb(sentence))

[]
[]
[]
[]
[]
[{'entity_group': 'LOC', 'score': 0.98820996, 'word': 'Louis Martin', 'start': 0, 'end': 12}]
[]
[]
[]
[{'entity_group': 'PER', 'score': 0.9939813, 'word': 'Jacques LeForgeron et moi', 'start': 0, 'end': 25}, {'entity_group': 'ACT', 'score': 0.98191005, 'word': 'forgé des épées de qualité', 'start': 31, 'end': 58}]
[{'entity_group': 'DESC', 'score': 0.7915242, 'word': 'avec une lame tranchante comme un rasoir', 'start': 0, 'end': 40}]
[{'entity_group': 'PER', 'score': 0.9504716, 'word': 'Le comte de la Roche', 'start': 0, 'end': 20}, {'entity_group': 'DESC', 'score': 0.6053759, 'word': 'demandé un blason distinctif', 'start': 22, 'end': 51}, {'entity_group': 'DESC', 'score': 0.58072966, 'word': 'fils', 'start': 60, 'end': 65}]
[{'entity_group': 'PER', 'score': 0.9554817, 'word': 'une épée ornée', 'start': 0, 'end': 14}, {'entity_group': 'ACT', 'score': 0.8961279, 'word': "d'une fleur de lys", 'start': 14, 'end': 33}]
[{'entity_group': 'PER', 'score': 0.9949763, 'word': '

In [ ]:
sub_entities(ner_jb, Bertrandus)

[{'entity_group': 'MISC',
  'score': 0.77501416,
  'word': "Journl d'un Villageois Médiéval",
  'start': 0,
  'end': 31},
 {'entity_group': 'LOC',
  'score': 0.9258664,
  'word': 'France',
  'start': 34,
  'end': 41},
 {'entity_group': 'DATE',
  'score': 0.9888371,
  'word': "10 avril 1365 Aujourd'hui",
  'start': 42,
  'end': 69},
 {'entity_group': 'DATE',
  'score': 0.528513,
  'word': 'le',
  'start': 70,
  'end': 73},
 {'entity_group': 'LOC',
  'score': 0.9932431,
  'word': 'St-Pierre-sous-Bois',
  'start': 133,
  'end': 153},
 {'entity_group': 'PER',
  'score': 0.99018055,
  'word': 'Jean Dupont',
  'start': 233,
  'end': 245},
 {'entity_group': 'PER',
  'score': 0.9653243,
  'word': 'Dupont',
  'start': 270,
  'end': 277},
 {'entity_group': 'DATE',
  'score': 0.9942385,
  'word': '12 avril 1365',
  'start': 338,
  'end': 352},
 {'entity_group': 'LOC',
  'score': 0.59629416,
  'word': 'Bourgogne',
  'start': 365,
  'end': 375},
 {'entity_group': 'PER',
  'score': 0.99522066,
  'wo